In [ ]:
import math
import csv
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("final project").getOrCreate()
sc = spark.sparkContext

In [ ]:
movies = sc.textFile(name='./movies.csv')
ratings = sc.textFile(name='./ratings.csv')

In [ ]:
# creates key value pair of userid, movieid, and rating
def user_rating_mapper(line):
    (userID, movieID, rating, timestamp) = line.split(',')
    return (userID, (movieID, float(rating))) # (userId, (movieId, rating))

# filter ignores first line in rating csv
ratings_rdd = ratings.filter(lambda line: not line.startswith("userId")).map(user_rating_mapper)

# groups rating by userid and calculate average of that 
def cal_avg_rating(group):
    # input (group) : [ (movieid, rating), ....]
    # outout : average
    length = len(group)
    total_rating =0.0

    for movieId, rating in group:
        total_rating = total_rating + rating

    return total_rating/length

# this rdd has represenation like [{"userid",avg, "userid2":avg, ....}]
user_avg = ratings_rdd.groupByKey().mapValues(cal_avg_rating)

# baselined user rating around zero 
def optimized(x):
    # input(x) : (userid, ((movieId, rating), average)) -> ('21', (('799', 5.0), 3.8285714285714287))
    # output: (userid, (movieId, rating - average))
    return (x[0], (x[1][0][0], x[1][0][1] - x[1][1]))

centered_ratings = ratings_rdd.join(user_avg).map(optimized)



In [ ]:
# Map movie ID, title, and genres 
def movie_genre_mapper(line):
    # you can't split using ',' becasue some movie title has a comman
    # you have to use csv reader to split the lines 
    fields = next(csv.reader([line])) 
    movieId = fields[0]
    title = fields[1]
    genres = fields[2].split('|')
    return (movieId, (title, genres))  # (movieId, (title, [genres]))


movies_rdd = movies.filter(lambda line: not line.startswith("movieId")).map(movie_genre_mapper)
movie_rating_rdd = centered_ratings.map(lambda x: (x[1][0],(x[0],x[1][1]))) # (movieId, (userId, rating))

# create key-value pair for user's rating to each genre they have rated
def genre_and_rating_mapper(x):
    # input(x) : (movieId, ((userId, rating), (title, [genres])))
    #input(example): ('327', (('72', -1.0238095238095237), ('Tank Girl (1995)', ['Action', 'Comedy', 'Sci-Fi'])))
    
    # output : [(userid, (genre,rating)), ...]
    genres = x[1][1][1]
    title = x[1][1][0]
    rating = x[1][0][1]
    userId = x[1][0][0]
    movieID = x[0]

    values = []

    for genre in genres:
        values.append( (userId, (genre,  rating)))

    return values


user_genre_scores = movie_rating_rdd.join(movies_rdd).flatMap(genre_and_rating_mapper)


# group by userID and find average of rated item user watched 
def avg_per_genre(group):
    #input: [ (genre, rating), .....(genre n, rating n)]
    #output: user profile {"genre": coefficent, ....} 
    total = {}
    count = {}
    for genre, rating in group:
        total[genre] = total.get(genre,0.0) + rating
        count[genre] = count.get(genre, 0) + 1

    avg = {}
    for g in total:
        avg[g] = total[g] / count[g]

    return avg


user_profile = user_genre_scores.groupByKey().mapValues(avg_per_genre)

In [ ]:
# create item profile 

# gather all genres
all_genres = movies_rdd.flatMap(lambda x: x[1][1]).distinct().collect()

# create item profile vector 
# e.g., [ (movieid, vector), ....]
# vector = movie profile = {"genre1":0.0, "genre2":1.0, .....} 
def genres_to_dict_vector(genres, all_genres):  

    vector = {}
    for genre in all_genres:
        if genre in genres:
            vector[genre]=1.0
        else:
            vector[genre]=0.0
    
    return vector
# it creates this following rdd data representation {"movieId":{profile}, ...., "movieIDn":{prifle}}
item_profile = movies_rdd.mapValues(lambda x: genres_to_dict_vector(x[1], all_genres))

In [ ]:
# collete user prfile and movie they watched as a map, then braodcast to all nodes. 

# collects all user and their profile as a map
# example {"user1":{profile}, "user2":{profile},....}
all_user_profile = user_profile.collectAsMap()

# groups all movies that user watched and turns that to a set.
# example, {"user1": {m1,m2,..,mn}, "user2":{m1,m2,..,mn},.....,}
watched_movies = ratings_rdd.map(lambda x: (x[0], x[1][0])).groupByKey().mapValues(set).collectAsMap()

#sends the value to all nodes and recreate rdd
user_profile_broadcast = sc.broadcast(all_user_profile)
watched_movies_broadcast = sc.broadcast(watched_movies)

# cosine helper function
def cosine_similarity(user_vector, item_vector):
    dot_product = sum(user_vector.get(g, 0) * item_vector.get(g, 0) for g in item_vector)
    user_magnitudes = math.sqrt(sum(v ** 2 for v in user_vector.values()))
    movie_magnitudes = math.sqrt(sum(v ** 2 for v in item_vector.values()))
    if user_magnitudes == 0 or movie_magnitudes == 0:
        return 0.0
    return dot_product / (user_magnitudes * movie_magnitudes)


user_id = '5'  # change as needed

# gets the user profile 
# example, {'Drama': 0.017104714226116226,'Comedy': -0.19858156028368773,.......,'Film-Noir': 1.4680851063829787}
profile = user_profile_broadcast.value.get(user_id, {})  

# gets all the movies that user watched 
# example, {'1041','1056','1060',........,'1090','1094','110'}
watched = watched_movies_broadcast.value.get(user_id, set())  

# filter out all movies that user watched, computes cosine similarity, and takes out top 10 movies. 
recommendations = item_profile \
    .filter(lambda x: x[0] not in watched) \
    .map(lambda x: (x[0], cosine_similarity(profile, x[1]))) \
    .takeOrdered(10, key=lambda x: -x[1])

for movieId, score in recommendations:
    print(f"Movie {movieId} -> similarity: {score:.4f}")



In [ ]:
# display user profile
profile

In [ ]:
# show movie user watched 
print(f"move he watched: ",len(watched))
watched

In [ ]:
# stop spark application 
sc.stop()